<a href="https://colab.research.google.com/github/dineshssdn-867/AI_Shakespeare/blob/master/AI_Shakespeare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [1]:
import tensorflow as tf
import numpy as np
import os

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')


1122304/1115394 [==============================] - 0s 0us/step


In [3]:
text = open(path_to_file, 'rb').read()
text = text.decode(encoding='utf-8')
print ('Total number of characters in the corpus is:', len(text))
print('The first 100 characters of the corpus are as follows:\n', text[:100])

Total number of characters in the corpus is: 1115394
The first 100 characters of the corpus are as follows:
 First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


# Vectorizing the Text

In [4]:
vocab = sorted(set(text))
print ('The number of unique characters in the corpus is', len(vocab))
print('A slice of the unique characters set:\n', vocab[:10])

The number of unique characters in the corpus is 65
A slice of the unique characters set:
 ['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3']


In [5]:
# Create a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
# Make a copy of the unique set elements in NumPy array format for later use in the decoding the predictions
idx2char = np.array(vocab)
# Vectorize the text with a for loop
text_as_int = np.array([char2idx[c] for c in text])

# Creating the Dataset

In [6]:
# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int) 
# for i in char_dataset.take(5): 
#   print(i.numpy())
seq_length = 100 # The max. length for single input
# examples_per_epoch = len(text)//(seq_length+1) # double-slash for “floor” division
sequences = char_dataset.batch(seq_length+1, drop_remainder=True) 
# for item in sequences.take(5): 
#   print(repr(''.join(idx2char[item.numpy()])))

In [7]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

In [8]:
BUFFER_SIZE = 10000 # TF shuffles the data only within buffers

BATCH_SIZE = 64 # Batch size

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

print(dataset)

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>


# Building the Model

In [9]:
# Length of the vocabulary in chars
vocab_size = len(vocab)
# The embedding dimension
embedding_dim = 256
# Number of RNN units
rnn_units = 1024

In [10]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [13]:
model = build_model(
    vocab_size = len(vocab), # no. of unique characters
    embedding_dim=embedding_dim, # 256
    rnn_units=rnn_units, # 1024
    batch_size=BATCH_SIZE)  # 64 for the traning

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
gru_1 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_1 (Dense)              (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


# Compiling and Training

In [14]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

# example_batch_loss  = loss(target_example_batch, example_batch_predictions)
# print("Prediction shape: ", example_batch_predictions.shape, " (batch_size, sequence_length, vocab_size)")
# print("scalar_loss:      ", example_batch_loss.numpy().mean())

model.compile(optimizer='adam', loss=loss)

In [16]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [17]:
EPOCHS = 30
history = model.fit(dataset, 
                    epochs=EPOCHS, 
                    callbacks=[checkpoint_callback])

Epoch 1/30
172/172 [==============================] - 18s 56ms/step - loss: 2.6609
Epoch 2/30
172/172 [==============================] - 11s 57ms/step - loss: 1.9480
Epoch 3/30
172/172 [==============================] - 11s 58ms/step - loss: 1.6824
Epoch 4/30
172/172 [==============================] - 11s 60ms/step - loss: 1.5356
Epoch 5/30
172/172 [==============================] - 11s 58ms/step - loss: 1.4492
Epoch 6/30
172/172 [==============================] - 11s 58ms/step - loss: 1.3903
Epoch 7/30
172/172 [==============================] - 11s 58ms/step - loss: 1.3457
Epoch 8/30
172/172 [==============================] - 11s 59ms/step - loss: 1.3070
Epoch 9/30
172/172 [==============================] - 11s 60ms/step - loss: 1.2711
Epoch 10/30
172/172 [==============================] - 11s 58ms/step - loss: 1.2382
Epoch 11/30
172/172 [==============================] - 11s 58ms/step - loss: 1.2059
Epoch 12/30
172/172 [==============================] - 11s 59ms/step - loss: 1.1737
E

# Generating New Text

In [18]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_30'

In [19]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
gru_2 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_2 (Dense)              (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [20]:
def generate_text(model, num_generate, temperature, start_string):
  input_eval = [char2idx[s] for s in start_string] # string to numbers (vectorizing)
  input_eval = tf.expand_dims(input_eval, 0) # dimension expansion
  text_generated = [] # Empty string to store our results
  model.reset_states() # Clears the hidden states in the RNN

  for i in range(num_generate): #Run a loop for number of characters to generate
    predictions = model(input_eval) # prediction for single character
    predictions = tf.squeeze(predictions, 0) # remove the batch dimension

    # using a categorical distribution to predict the character returned by the model
    # higher temperature increases the probability of selecting a less likely character
    # lower --> more predictable
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # The predicted character as the next input to the model
    # along with the previous hidden state
    # So the model makes the next prediction based on the previous character
    input_eval = tf.expand_dims([predicted_id], 0) 
    # Also devectorize the number and add to the generated text
    text_generated.append(idx2char[predicted_id]) 

  return (start_string + ''.join(text_generated))

In [23]:
generated_text = generate_text(
                    model, 
                    num_generate=500, 
                    temperature=1, 
                    start_string=u"Insomnia")
print(generated_text)

Insomniate,--I be heark'd with knock;
The King of Norfolk, Thomas Mowbray,
Have wrong yourself with unsturing water: I say it
should be constant in again.

Pedant:
Ay, sir, withyou tread the case, your ears never parden him, your name
Is not his harm.

VIRGINIUS:
Blessed up; when?
Nay, let them forth to crave.

Nurse:
His name? a know! more much cozerets up,
Is guilty of: but when I have been frea,
And not thyself we were u hasty Coriolanus!

CLARENCE:
I willingly as if at me for
his treasons of the rea
